In [2]:
using Pipe

In [77]:
normInf(vec::Tuple{Int64, Int64}) =
  begin
    # return map((x->x*x), vec) |> sum |> sqrt
    # return map(abs, vec) |> sum
    return map(abs, vec) |> Base.splat(max)
  end

norm1(vec::Tuple{Int64, Int64}) =
  begin
    return map(abs, vec) |> sum
  end

normalize(vec::Tuple{Int64, Int64}) =
  begin
    size = normInf(vec)
    if (size == 0)
      return vec
    end
    return map(x -> div(x, size), vec)
  end

displayKnots(knots::Vector{Tuple{Int64, Int64}}) =
  begin
    nRows = 26
    nCols = 26
    # board = Matrix{Char}('_', nRows, nCols)
    board = fill('_', (nRows, nCols))    

    for i in length(knots):-1:1
      (x, y) = knots[i]
      board[div(nRows,2)-y, div(nCols,2)+x+1] = '0' + i - 1 
    end

    stringFormVec=Vector{String}()
    for i in 1:nRows
      strToCreate = ""
      for j in 1:nCols
        strToCreate *= board[i,j]
      end
      push!(stringFormVec, strToCreate)
    end

    stringForm = map(string, board)
    display(stringFormVec)
  end

displayKnots (generic function with 1 method)

In [6]:
offsetFromDirection = Dict('U' => (0, 1), 'D' => (0,-1), 'L' => (-1,0), 'R' => (1,0))

parseInputLine(line::String) =
  begin
    splitted = split(line)
    direction = splitted[1] |> first
    (dx, dy) = offsetFromDirection[direction]
    count = parse(Int64, splitted[2])
    return (count*dx, count*dy)
  end

parseInputLine (generic function with 1 method)

In [44]:
moves = Vector{Tuple{Int64, Int64}}()
for move in eachline("test2.txt")
  @pipe move |> parseInputLine |> push!(moves, _)
end

In [22]:
# task 1
reachedPositions = Set{Tuple{Int64, Int64}}([(0,0)])
tPos = hPos = (0,0)

for move in moves
  # willMoveTowards = @pipe move |> map(sign, _) |> indexin(x-> x == 0, _) |> 3 - _
  (dx, dy) = normalize(move)
  for _ in 1:norm(move)
    newHPos = hPos .+ (dx, dy)
    if norm(tPos .- newHPos) > 1
      tPos = hPos
      push!(reachedPositions, tPos)
    end
    hPos = newHPos
  end
end

length(reachedPositions)

6332

In [78]:
# task 2
reachedPositions = Set{Tuple{Int64, Int64}}([(0,0)])
knotsCount = 10
knotPositions = Vector{Tuple{Int64, Int64}}()
newKnotPositions = Vector{Tuple{Int64, Int64}}()
for _ in 1:knotsCount
  push!(knotPositions, (0,0))
  push!(newKnotPositions, (0,0))
end

for move in moves
  displayKnots(knotPositions)
  (dx, dy) = normalize(move)
  for _ in 1:normInf(move)
    # displayKnots(knotPositions)
    newKnotPositions[1] = knotPositions[1] .+ (dx, dy)
    
    for i in 1:knotsCount-1
      if normInf(knotPositions[i+1] .- newKnotPositions[i]) > 1
        newKnotPositions[i+1] = knotPositions[i]
        # println(i, "-", newKnotPositions[i+1])
        if norm1(knotPositions[i+1] .- newKnotPositions[i]) > 2
          newKnotPositions[i+1] = newKnotPositions[i+1] .+ (dx, dy)
          # println(i, "!!!-", newKnotPositions[i+1])
        end
        if newKnotPositions[i+1] == newKnotPositions[i] || normInf(newKnotPositions[i+1] .- knotPositions[i+1]) > 1
          # newKnotPositions[i+1] = newKnotPositions[i+1] .- (dx, dy)
          newKnotPositions[i+1] = knotPositions[i]
        end
      else newKnotPositions[i+1] = knotPositions[i+1]
      end
    end

    knotPositions = copy(newKnotPositions)
    push!(reachedPositions, knotPositions[knotsCount])
  end
end

reachedPositions

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________0____________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________543210_______"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________0_______"
 "__________________1_______"
 "__________________2_______"
 "__________________3_______"
 "_________________54_______"
 "________________6_________"
 "_______________7__________"
 "______________8___________"
 "_____________9____________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________01234___________"
 "______________5___________"
 "_______________6__________"
 "_______________7__________"
 "______________98__________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "___________2345___________"
 "__________1____6__________"
 "__________0____7__________"
 "______________98__________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

LoadError: BoundsError: attempt to access 26×26 Matrix{Char} at index [8, 27]

In [15]:
findfirst(x-> x == 1, [2,4]) == 1

false

In [21]:
reachedPositions

Set{Tuple{Int64, Int64}} with 6332 elements:
  (194, -18)
  (97, 68)
  (345, -50)
  (125, 10)
  (118, 24)
  (251, 21)
  (266, 14)
  (60, -78)
  (363, -65)
  (93, -62)
  (185, -25)
  (353, 18)
  (58, -61)
  (131, -8)
  (196, -20)
  (72, -87)
  (288, 9)
  (324, -20)
  (18, -6)
  (329, -7)
  (88, 80)
  (117, -74)
  (206, 0)
  (183, -25)
  (161, -41)
  ⋮ 

In [18]:
for i in 10:-1:1
  display('0' + i - 1) 
end

'9': ASCII/Unicode U+0039 (category Nd: Number, decimal digit)

'8': ASCII/Unicode U+0038 (category Nd: Number, decimal digit)

'7': ASCII/Unicode U+0037 (category Nd: Number, decimal digit)

'6': ASCII/Unicode U+0036 (category Nd: Number, decimal digit)

'5': ASCII/Unicode U+0035 (category Nd: Number, decimal digit)

'4': ASCII/Unicode U+0034 (category Nd: Number, decimal digit)

'3': ASCII/Unicode U+0033 (category Nd: Number, decimal digit)

'2': ASCII/Unicode U+0032 (category Nd: Number, decimal digit)

'1': ASCII/Unicode U+0031 (category Nd: Number, decimal digit)

'0': ASCII/Unicode U+0030 (category Nd: Number, decimal digit)

In [20]:
norm1((0,0) .- (2,0))

2

In [57]:
string('a')

"a"

In [74]:
"asd" * 'f'

"asdf"